# BFGS (Broyden, Fletcher, Goldfarb and Shanno)

Quasi-Newton method which approximates the Hessian at each step. We reset the Hessian to I if curvature condition fails.

In [5]:
import matplotlib.pyplot as plt
import autograd.numpy as np
from autograd import jacobian
from autograd import elementwise_grad

In [6]:
#takes a numpy array
def f(x):
    return np.array([np.exp(x[0])+np.exp((-1)*x[0]+4.) + x[1]**2])
    #return (x-7)*(x-3)

In [41]:
def MSE(y,x):
    return np.linalg.norm(y-x)

In [42]:
# general class of linesearch iterative methods for continuous optimisation
class LineSearch:
    
    def __init__(self,backtracking,step):
        self.backtracking = backtracking
        self.step = step
        
        self.sigma = 0.1
        self.tol = 0.00000001
        self.rho = 1/2
        
    
    def Armijo_test(self,f,x,step,direction):
        a = f(x+step*direction)
        b = f(x) - self.sigma*step*np.dot(direction,direction)
        return (a < b).all()
    
    #starts from previous step and finds suitable step length by testing for the Armijo condition
    def find_step(self,f,x,step,direction):
        iters =0
        
        while not self.Armijo_test(f,x,step,direction):
            step = self.rho * step
            iters +=1
            if iters > 10000:
                return None
        return step
        
    #minimise f from starting point x
    def find_minimum(self,f,x):
        #executes steps until convergence
        prev_x = 10*10*10
        iters = 0
        step = self.step
        
        grad = elementwise_grad(f)(x)
        #set initial direction to steepest descent
        direction = np.negative(grad)
        #set initial hessian to I so that first step is gradient descent
        hessian = np.identity(x.size)
        
        
        while np.linalg.norm(x-prev_x) > self.tol:
            
            prev_grad = grad
            grad = elementwise_grad(f)(x)
            delta_grad = grad - prev_grad
            delta_x = x - prev_x
            
            hessian = BFGS(f,delta_x,delta_grad, hessian)
            direction = np.dot(np.linalg.inv(hessian),np.negative(grad))
            
            if self.backtracking :
                step = self.find_step(f,x,step,direction)
            if step == None:
                print("step not found")
                break

            prev_x = x
            
            #we use previous direction to compute the next            
            x = x + step*direction
            
            if iters > 10000:
                print("iteration limit reached")
                return
            else:
                #if iters % 10 == 0:
                print("progress")
                print(x)
                print(f(x))
                print()
                iters +=1
        print("converged in %d iterations to" %iters)
        print("x value :", x)
        print("out : ", f(x))

In [43]:
# ||| x float |||
def BFGS(f,delta_x,delta_g, prev_hessian):
    if np.dot(delta_x.T,delta_g) > 0:
        q1 = np.dot(delta_g,delta_g.T)/np.dot(delta_g.T,delta_g)

        dxHessian = np.dot(delta_x.T, prev_hessian)

        q2 = np.dot(np.dot(prev_hessian,delta_x),dxHessian)/np.dot(dxHessian,delta_x)

        new_hessian = prev_hessian + q1 - q2
        return new_hessian
    else:
        return np.identity(delta_x.size)

In [44]:
bfgs = LineSearch(step = 100.,backtracking=True)

In [45]:

start = np.ones(10)
y = np.array([1.,21.,32.,43.,54.,65.,76.,87.,98.,109.])
def MSE_y(x):
    return MSE(y,x)

bfgs.find_minimum(MSE_y,start)


progress
[ 1.         10.52121907 15.75788956 20.99456004 26.23123053 31.46790102
 36.70457151 41.94124199 47.17791248 52.41458297]
110.05713508471928

progress
[  1.          20.04243814  30.51577911  40.98912009  51.46246106
  61.93580204  72.40914301  82.88248399  93.35582496 103.82916594]
10.057135084719286

progress
[  1.          21.23259052  32.36051531  43.48844009  54.61636488
  65.74428966  76.87221445  88.00013924  99.12806402 110.25598881]
2.4428649152807083

progress
[  1.          20.93505242  31.89933126  42.86361009  53.82788892
  64.79216776  75.75644659  86.72072542  97.68500426 108.64928309]
0.6821350847192944

progress
[  1.          21.00943695  32.01462727  43.01981759  54.02500791
  65.03019823  76.03538856  87.04057888  98.0457692  109.05095952]
0.0991149152807002

progress
[  1.          21.00013888  32.00021527  43.00029165  54.00036804
  65.00044442  76.00052081  87.0005972   98.00067358 109.00074997]
0.0014586652806928555

progress
[  1.          20.9999936 